In [ ]:
import pandas as pd

# ファイルパス
files = {
    "FTSE_WGBI": '/home/shingo/Investment/file/ftse_wgbi_20250123.csv',
    "Gold": '/home/shingo/Investment/file/gold_20250123.csv',
    "All_Country": '/home/shingo/Investment/file/all_country_20250123.csv'
}

# データフレームを保存する辞書
dataframes = {}

# 各ファイルを処理
for name, path in files.items():
    try:
        # CSVファイルを読み込む（Shift_JISエンコーディングを指定）
        df = pd.read_csv(path, encoding='shift_jis')
        print(f"{name} のデータ読み込み成功。データサンプル:\n{df.head()}\n")  # デバッグ用
        
        # 必要な列を確認して抽出
        if '年月日' not in df.columns or '基準価額(円)' not in df.columns:
            raise ValueError(f"{name} に必要な列（年月日, 基準価額(円)）が存在しません。列名: {df.columns}")

        # 年月日を日付形式に変換（フォーマットを明示）
        df['年月日'] = pd.to_datetime(df['年月日'], format='%Y年%m月%d日', errors='coerce')
        
        # 必要な列を抽出
        df = df[['年月日', '基準価額(円)']].dropna()
        df.columns = ['年月日', name]  # 列名を統一
        
        # 年月日をインデックスに設定
        df.set_index('年月日', inplace=True)
        
        # 辞書に保存
        dataframes[name] = df

    except Exception as e:
        print(f"{name} の処理中にエラーが発生しました: {e}")

# データを統合（すべてのデータを含むように outer join を指定）
if dataframes:
    merged_data = pd.concat(dataframes.values(), axis=1, join='outer')
    print(f"統合データ:\n{merged_data.head()}\n")  # デバッグ用
else:
    print("データが統合できませんでした。すべてのファイルでエラーが発生した可能性があります。")

# CSVに保存する場合
output_path = '/home/shingo/Investment/file/merged_data.csv'
if 'merged_data' in locals() and not merged_data.empty:
    merged_data.to_csv(output_path, encoding='shift_jis')  # 日本語環境では utf-8-sig 推奨
    print(f"統合データが {output_path} に保存されました。")
else:
    print("統合データが空のため、保存されませんでした。")